In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import numpy as np # linear algebra
import pandas as pd # data processing


import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, save
import plotly.graph_objects as go
import sklearn
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
pd.plotting.register_matplotlib_converters()

In [2]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [3]:
train_df.drop('Province_State',axis=1,inplace=True)
train_df['Date']=pd.to_datetime(train_df['Date'])

In [4]:
test_df['Province_State']=test_df.drop('Province_State',axis=1)
test_df['Date']=pd.to_datetime(test_df['Date'])

In [5]:
# 문자를 숫자, 숫자를 문자로 맵핑
from sklearn.preprocessing import LabelEncoder

# 라벨 인코더 생성
LE=LabelEncoder()

# train데이터의 Country Region을 피팅하고 라벨숫자로 변환한다
train_df['Country_Region']=LE.fit_transform(train_df['Country_Region'])
test_df['Country_Region']=LE.transform(test_df['Country_Region'])

#날짜 정보를 문자열로 바꿔주는 strftime() 메서드 사용
train_df.loc[:, 'Date'] = train_df.Date.dt.strftime("%m%d")
#다시 int형으로 변환
train_df["Date"]  = train_df["Date"].astype(int)

test_df.loc[:, 'Date'] = test_df.Date.dt.strftime("%m%d")
test_df["Date"]  = test_df["Date"].astype(int)
test_df

,ForecastId,Province_State,Country_Region,Date
0,1,1,0,326
1,2,2,0,327
2,3,3,0,328
3,4,4,0,329
4,5,5,0,330
...,...,...,...,...
13153,13154,13154,179,503
13154,13155,13155,179,504
13155,13156,13156,179,505
13156,13157,13157,179,506


In [6]:
#Select feature column names and target variable we are going to use for training
# train 데이터로 들어갈 피처와 target피처를 설정
features=['Date','Country_Region']
target = 'ConfirmedCases'

#This is input which our classifier will use as an input.
train_df[features].head(10)

,Date,Country_Region
0,122,0
1,123,0
2,124,0
3,125,0
4,126,0
5,127,0
6,128,0
7,129,0
8,130,0
9,131,0


In [7]:
from sklearn.ensemble import RandomForestClassifier

# 모델 정의
rfcla = RandomForestClassifier(n_estimators=100, max_samples=0.8, random_state=1, n_jobs=-1)

# 학습
rfcla.fit(train_df[features],train_df[target])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=0.8,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [8]:
#Make predictions using the features from the test data set
predictions = rfcla.predict(test_df[features])

predictions

array([ 94., 110., 110., ...,   9.,   9.,   9.])

In [9]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'n_estimators'      : [100,200,300],
    'max_depth'         : [6, 8, 10, 12, 14],
    'min_samples_leaf'  : [1, 3, 5, 7]
}

rf_grid = GridSearchCV(rfcla, param_grid=parameters, scoring="accuracy", n_jobs=-1, verbose=1)
rf_grid.fit(train_df[features],train_df[target])

print("최고 평균 정확도 : {0: .4f}".format(rf_grid.best_score_))
printf("최고 파라미터 : ", rf_grid.best_params_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.6min


MemoryError: Unable to allocate 57.0 MiB for an array with shape (4529, 1, 1651) and data type float64